All the backends supported are as follow: 
- ProjectQ simulator
- Aer simulators (statevector, QASM, and unitary)
- IBMQ devices
- Rigetti QCS devices
- Rigetti QVM (for device simulation or statevector)
- AQT devices
- Honeywell devices
- Q# simulators

In [2]:
import numpy as np
from pytket import Circuit, OpType
from pytket.qiskit import qiskit_to_tk

from qiskit.quantum_info.operators import Operator
from qiskit import QuantumCircuit

In [3]:
SEARCHED_STRING = "11"

N = len(SEARCHED_STRING)
oracle = np.zeros(shape=(2 ** N, 2 ** N))
for b in range(2 ** N):
    if np.binary_repr(b, N) == SEARCHED_STRING:
        oracle[b, b] = -1
    else:
        oracle[b, b] = 1
        
diffusion = 2.0 * np.full((2 ** N, 2 ** N), 1 / (2 ** N)) - np.eye(2 ** N)        

#print(oracle)
#print(diffusion)

In [4]:
circuit = QuantumCircuit(N)

for i in range(N):
    circuit.i(i)

for i in range(N):
    circuit.h(i)

In [5]:
oracle_gate = Operator(oracle)
diffusion_gate = Operator(diffusion)

In [6]:
N_ITER = int(np.pi / 4 * np.sqrt(2 ** N))

In [7]:
oracle_diffusion_qubit_targets = []

for i in range(N):
    oracle_diffusion_qubit_targets.append(i)
    
for i in range(N_ITER):
    circuit.unitary(oracle_gate, oracle_diffusion_qubit_targets, label='oracle_gate')

    circuit.unitary(diffusion_gate, oracle_diffusion_qubit_targets, label='diffusion_gate')

In [13]:
circ = qiskit_to_tk(circuit)

In [14]:
from pytket.backends.ibm import AerStateBackend, AerBackend, AerUnitaryBackend, IBMQBackend

In [15]:
circ.measure_all()

[noop q[0]; noop q[1]; H q[0]; H q[1]; Unitary2qBox q[0], q[1]; Unitary2qBox q[0], q[1]; Barrier q[0], q[1]; Measure q[0] --> meas[0]; Measure q[1] --> meas[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [17]:
import time
start_time = time.time()

aer_b = AerBackend()
shots_handle = aer_b.process_circuit(circ, n_shots=10, seed=1)

print(time.time() - start_time)

shots = aer_b.get_shots(shots_handle)
print(shots)

0.009415864944458008
[[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
